In [1]:
from collections import defaultdict, Counter, deque
from intervaltree import Interval, IntervalTree
import itertools
import functools
import math
import numpy as np
import re
from typing import Callable, TypeVar

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [47]:
def day5_maps(data):
    names, maps = [], []

    def create_map(lines):
        ranges = IntervalTree([Interval(0, math.inf)])
        name = lines[0].split()[0]
        for i, line in enumerate(lines[1:]):
            if not line:
                break
            dest, source, size = map(int, line.split())
            ranges.chop(source, source+size)
            ranges[source:source+size] = dest
        return name, ranges, lines[i+2:]

    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]

    while len(data):
        name, intervals, data = create_map(data)
        names.append(name)
        maps.append(intervals)

    return list(seeds), maps, names

seeds, intervals, names = day5_maps(data(5))

def day5_1(seeds, data):
    lowest = math.inf
    for seed in seeds:
        for interval in data:
            start, end, dest = next(iter(interval.at(seed)))
            seed = seed if dest is None else (seed-start)+dest
        lowest = min(lowest, seed)
    return lowest

def day5_2(seeds, data):
    seedrange = IntervalTree([Interval(start, start+size) for start, size in zip(seeds[::2], seeds[1::2])])
    seedrange.merge_overlaps()

    for i, interval in enumerate(data):
        newrange = IntervalTree()
        # print(names[i], seedrange, interval)
        for (seedstart, seedend, _) in seedrange:
            intersections = interval[seedstart:seedend]
            for (start, end, dest) in intersections:
                newstart = (dest + max(seedstart, start)-start) if dest is not None else seedstart
                newend = (dest + min(seedend, end)-start) if dest is not None else seedend
                newrange.addi(newstart, newend)
        newrange.merge_overlaps()
        seedrange = newrange
        
    lowest = min([loc[0] for loc in seedrange])
    return lowest

day5_1(seeds, intervals), day5_2(seeds, intervals)


(1181555926, 37806486)

In [38]:
data6 = data(6, lambda x: list(map(int, x.split(':')[-1].split())))

def day6(times, distances):
    result = []
    for t, d in zip(times, distances):
        # (t-x)*x = d+1
        # -x**2 +xt -d-1 = 0
        a, b, c = -1, t, -d-1
        det = math.sqrt(b**2 - 4*a*c)
        if det < 0:
            continue
        zeros = (-b+det)/(2*a), (-b-det)/(2*a)
        floor, ceil = math.ceil(min(zeros)), math.floor(max(zeros))
        result.append(ceil-floor+1)
    return functools.reduce(lambda x, y: x*y, result, 1)

day6(*data6), day6(*[[int(''.join(map(str, x)))] for x in data6])

(252000, 36992486)

In [142]:
data7 = data(7, lambda x: x.split())

def card_ord(card, flag=False):
    match card:
        case 'T': return 10
        case 'J': return 1 if flag else 11
        case 'Q': return 12
        case 'K': return 13
        case 'A': return 14
    return int(card)

def score_hand(hand, flag=False):
    cards = [card_ord(card, flag) for card in hand]
    if flag:
        filtered = Counter(hand.replace('J', ''))
        hand = hand.replace('J', filtered.most_common(1)[0][0] if filtered else 'A')

    counts = Counter(hand)
    score = sum([counts[x]**2 for x in counts])
    return [score]+cards

def day7(data, flag=False):
    key = lambda x: score_hand(x[0], flag)
    ordered = sorted(data, key=key)
    print(ordered)
    return sum((i+1)*int(score) for i, (_, score) in enumerate(ordered))

day7(data7), day7(data7, True)

[['234T5', '953'], ['239TA', '384'], ['24798', '565'], ['248T6', '561'], ['257KA', '932'], ['25JQ7', '667'], ['26QA5', '195'], ['26K9T', '351'], ['27598', '271'], ['276A9', '29'], ['27J9T', '416'], ['29Q3K', '735'], ['2T5J9', '68'], ['2T5K9', '803'], ['2T9JQ', '651'], ['2J367', '487'], ['2JT7A', '398'], ['2JQT5', '296'], ['2QT94', '553'], ['2QA58', '791'], ['2K584', '785'], ['2KJ34', '151'], ['2A8Q6', '270'], ['3628Q', '25'], ['36KA7', '972'], ['37J4A', '607'], ['38KT2', '262'], ['3J95K', '470'], ['3Q27A', '691'], ['3Q298', '20'], ['3K7J6', '299'], ['3K857', '563'], ['4253A', '712'], ['456JK', '958'], ['45A79', '217'], ['465J8', '386'], ['46T8J', '628'], ['46A5J', '554'], ['483A2', '836'], ['4965Q', '431'], ['49A2Q', '241'], ['4J295', '501'], ['4J2T9', '330'], ['4J789', '687'], ['4J7KT', '744'], ['4A5Q6', '247'], ['5483K', '126'], ['5489J', '363'], ['54A26', '692'], ['5679J', '143'], ['57KAJ', '949'], ['57A9Q', '879'], ['5JQT6', '504'], ['5JA34', '872'], ['5JA86', '725'], ['5Q683', '79

(248812215, 250057090)

In [69]:
def parse8(data):
    instructions, nodes = data[0], {}
    for line in data[2:]:
        start, l, r = re.findall(r'(.{3}) = \((.{3}), (.{3})\)', line)[0] 
        nodes[start] = (l, r)
    return instructions, nodes

data8 = parse8(data(8))

def day8(instructions, nodes, start, ends):
    count, node = 0, start
    for i in itertools.cycle(instructions):
        node = nodes[node][0 if i == 'L' else 1]
        count +=1
        if ends(node):
            break
    return count

day8(*data8, 'AAA', lambda x: x == 'ZZZ')

18673

In [70]:
def day8_2(instructions, nodes):
    results = []
    for start in filter(lambda x: re.match(r'..A', x), nodes):
        results.append(day8(instructions, nodes, start, lambda x: re.match(r'..Z', x)))
    return math.lcm(*results)

day8_2(*data8)

17972669116327

In [44]:
data9 = data(9, lambda line: [int(x) for x in line.split()])

def determine_steps(arr, i=-1):
    steps = [arr[i]]
    while len(np.unique(arr)) > 1:
        arr = np.diff(arr)
        steps.append(arr[i])
    return steps

def day9(data, part2=False):
    part1, part2 = 0, 0
    for history in data:
        steps = determine_steps(np.array(history)), determine_steps(np.array(history), 0)
        part1 += sum(steps[0])
        part2 += sum(steps[1][::2]) - sum(steps[1][1::2])
    return part1, part2

day9(data9)

In [71]:
data10 = np.array(data(10, list))

pipe_map = {
    '|': [(0, -1), (0, 1)],
    '-': [(-1, 0), (1, 0)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, -1), (-1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, 1), (1, 0)],
}

def pipe_neighbors(pipe, x, y):
    if pipe not in pipe_map:
        return
    for i in pipe_map[pipe]:
        yield (x+i[0], y+i[1])

def get_neighbors(x, y):
    return [(x-1, y), (x+1, y), (x, y+1), (x, y-1)]

def fix_pipe_start(arr):    
    start_coords = np.where(arr == 'S')
    y, x = start_coords[0][0], start_coords[1][0]

    neighbors = []
    for i, j in get_neighbors(x, y):
        if (x, y) in pipe_neighbors(data10[j, i], i, j):
            neighbors.append((i-x, j-y))
    for pipe in pipe_map.keys():
        if frozenset(pipe_map[pipe]) == frozenset(neighbors):
            arr[y, x] = pipe
    return x, y

def dijkstra(nodes, start, get_neighbors, end_condition=lambda _: False):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft()
        if end_condition(current):
            return visited, current
        if current in visited:
            continue
        for node in get_neighbors(nodes, current):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

def day10(data):
    arr = data.copy()
    start = fix_pipe_start(arr)
    # print(arr)
    visited, _ = dijkstra(data, start, lambda _, coords: pipe_neighbors(arr[coords[1], coords[0]], *coords))

    visual = data.copy()
    for x, y in visited:
        visual[y, x] = visited[(x, y)]
    print(visual)

# max(visited.values())
    return visited

visited = day10(data10)

[['7' '.' '7' '7' 'F' '7' 'F' '|' '-' 'F' '.' 'J' '-' 'J' '7' '-' 'L' 'F' '|' '-' '7' '.' 'F' 'F' 'L' '7' 'F' '-' 'L' '-' ... '-' '7' '-' 'L' 'J' '7' '7' 'F' '7' '-' 'F' '-' 'F' 'J' '7' '7' '.' '7' '7' 'J' '.' 'J' '7' '7' 'F' '-' 'L' '7' '7' '.']
 ['F' '-' 'F' '-' 'J' '|' 'F' 'L' '-' 'J' '7' '-' 'L' '|' '.' 'L' '|' 'F' 'J' '|' 'L' 'F' '-' '7' 'J' 'L' '|' 'J' '.' '|' ... 'F' 'F' '.' '|' '-' 'J' '7' 'L' '-' '-' '|' '.' '.' '|' '-' 'F' '-' '-' 'J' '|' '|' 'L' 'F' '-' '|' '7' '.' '|' '-' '.']
 ['F' 'F' '-' 'J' 'F' '7' '7' '.' 'L' '7' '-' '-' '7' 'F' '-' '-' 'J' 'L' 'F' '|' 'L' 'L' '-' 'L' '-' '.' '7' '-' '|' 'J' ... '7' '.' '.' 'F' 'J' '|' '|' '|' 'L' '7' 'F' '7' '7' '.' 'F' 'L' '-' '|' '7' '7' 'J' 'L' '|' '-' 'F' 'F' '-' '|' '|' '7']
 ['|' '.' '|' '|' 'L' '7' 'J' 'F' 'L' 'F' '7' 'L' '7' '.' '|' 'J' '|' 'L' 'L' 'J' '7' 'J' 'F' '-' 'J' 'L' '-' '7' '|' '7' ... '.' 'F' 'F' '|' 'F' '|' '|' 'F' '|' '.' '|' '|' 'L' 'F' '-' 'L' '-' '|' 'J' 'L' '7' 'F' 'L' '.' 'L' 'L' 'F' '-' '7' '7']
 ['7' 'F' '|

In [70]:
claimed = set()
for y, row in enumerate(data10):
    inside = False
    for x, pipe in enumerate(row):
        if (x, y) in visited:
            if pipe in 'LJ|':
            # print(y, x, (x, y) in visited, pipe)
                inside ^= True # If pipe is part of path and is north: taken from https://old.reddit.com/r/adventofcode/comments/18fgddy/2023_day_10_part_2_using_a_rendering_algorithm_to/
        elif inside:
            claimed.add((x, y))

test = data10.copy()
for x, y in claimed:
    test[y, x] = 'I'
print(test, len(claimed))

[['F' 'F' '7' 'F' 'S' 'F' '7' 'F' '7' 'F' '7' 'F' '7' 'F' '7' 'F' '-' '-' '-' '7']
 ['L' '|' 'L' 'J' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' '|' 'F' '-' '-' 'J']
 ['F' 'L' '-' '7' 'L' 'J' 'L' 'J' '|' '|' '|' '|' '|' '|' 'L' 'J' 'L' '-' '7' '7']
 ['F' '-' '-' 'J' 'F' '-' '-' '7' '|' '|' 'L' 'J' 'L' 'J' 'I' 'F' '7' 'F' 'J' '-']
 ['L' '-' '-' '-' 'J' 'F' '-' 'J' 'L' 'J' 'I' 'I' 'I' 'I' 'F' 'J' 'L' 'J' 'J' '7']
 ['|' 'F' '|' 'F' '-' 'J' 'F' '-' '-' '-' '7' 'I' 'I' 'I' 'L' '7' 'L' '|' '7' '|']
 ['|' 'F' 'F' 'J' 'F' '7' 'L' '7' 'F' '-' 'J' 'F' '7' 'I' 'I' 'L' '-' '-' '-' '7']
 ['7' '-' 'L' '-' 'J' 'L' '7' '|' '|' 'F' '7' '|' 'L' '7' 'F' '-' '7' 'F' '7' '|']
 ['L' '.' 'L' '7' 'L' 'F' 'J' '|' '|' '|' '|' '|' 'F' 'J' 'L' '7' '|' '|' 'L' 'J']
 ['L' '7' 'J' 'L' 'J' 'L' '-' 'J' 'L' 'J' 'L' 'J' 'L' '-' '-' 'J' 'L' 'J' '.' 'L']] 10


In [ ]:
claimed = set()


In [280]:
test = day10(data10)

Fs = [(x*2, y*2) for x, y in test if data10[y, x] == 'F']

def fill_neighbors(arr, x, y):
    west, east, south, north = get_neighbors(x, y)
    if west[0] < 0 or north[1] < 0 or east[0] > len(arr[0])-1 or south[1] > len(arr)-1:
        return

    yield from [(x, y) for x, y in [west, east, south, north] if arr[y, x] == '.']

    # lr, ns = '|.', '-.'
    # for (x, y), allowed in ((west, lr), (east, lr), (south, ns), (north, ns)):
    #     p = data10[y, x]
    #     if p not in allowed:
    #         yield '!'
    #     if p == '.':
    #         yield (x, y)

def blowup(arr):
    new = np.full((arr.shape[0]*2, arr.shape[1]*2), '.')
    for y, row in enumerate(arr):
        for x, i in enumerate(row):
            new[y*2, x*2] = i
            new[y*2, x*2+1] = '-' if i in '-SFL' else '.'
            new[y*2+1, x*2] = '|' if i in '|SF7' else '.'
    return new

expanded = blowup(data10)
# print(expanded)
# print(Fs)

checked = set()
result = 0
filled = data10.copy()
for (fx, fy) in Fs:
    if (fx, fy) in checked:
        continue
    visited, failed = dijkstra(expanded, (fx+1, fy+1), lambda _, coords: fill_neighbors(expanded, *coords), lambda x: x[0] in (0, len(expanded[0])-1) or x[1] in (0, len(expanded)-1))
    b = expanded.copy()
    for (x, y) in visited:
        b[y, x] = '0'
    # print(b, failed, len(visited), visited)

    if not failed:
        hmm, q = dijkstra(data10, (fx//2+1, fy//2+1), lambda _, coords: fill_neighbors(data10, *coords))
        for (x, y) in hmm:
            filled[y, x] = '0'
        result += len(hmm)

    checked = checked.union(set(visited.keys()))
print(filled, result)

# TODO: 
# Fs are not the only places to start checking. a | could be diagnol from an F. Alternatively, it could be covered by something like F->|-
# Example doesn't seem to be terminating properly
# Maybe just use an entirely different approach?

[['.' 'F' '-' '-' '-' '-' '7' 'F' '7' 'F' '7' 'F' '7' 'F' '-' '7' '.' '.' '.' '.']
 ['.' '|' '0' '-' '-' '7' '|' '|' '0' '|' '0' '|' '0' '|' '0' 'J' '.' '.' '.' '.']
 ['.' '|' '|' '.' 'F' 'J' '|' '|' '|' '|' '|' '|' '|' '|' 'L' '7' '.' '.' '.' '.']
 ['F' 'J' 'L' '7' 'L' '0' 'L' 'J' 'L' 'J' '|' '|' 'L' 'J' '.' 'L' '-' '7' '.' '.']
 ['L' '0' '-' 'J' '.' 'L' '7' '.' '.' '.' 'L' 'J' 'S' '7' 'F' '-' '7' 'L' '7' '.']
 ['0' '0' '0' '0' 'F' '-' 'J' '.' '.' 'F' '7' 'F' 'J' '|' 'L' '7' 'L' '7' 'L' '7']
 ['0' '0' '0' '0' 'L' '0' '0' 'F' '7' '|' '|' 'L' '7' '|' '.' 'L' '7' 'L' '7' '|']
 ['0' '0' '0' '0' '0' '|' 'F' 'J' 'L' 'J' '|' 'F' 'J' '|' 'F' '7' '|' '.' 'L' 'J']
 ['0' '0' '0' '0' 'F' 'J' 'L' '-' '7' '.' '|' '|' '.' '|' '|' '|' '|' '.' '.' '.']
 ['.' '0' '0' '0' 'L' '0' '-' '-' 'J' '.' 'L' 'J' '.' 'L' 'J' 'L' 'J' '.' '.' '.']] 30


In [66]:
data11 = np.array(data(11, list))

def day11(data, expansions):
    cols, rows = [IntervalTree([Interval(x, x+1) for x in np.where(np.all(data == '.', axis=i))[0]]) for i in range(2)]

    galaxies = np.where(data == '#')

    results = Counter()
    for a, b in itertools.combinations(range(len(galaxies[0])), 2):
        xs, ys = Interval(galaxies[0][a], galaxies[0][b]), Interval(*sorted([galaxies[1][a], galaxies[1][b]]))
        empties = len(cols[ys[0]:ys[1]]) + len(rows[xs[0]:xs[1]])
        distance = ys[1]-ys[0] + xs[1]-xs[0]
        for expansion in expansions:
            results[expansion] += distance + empties * (expansion-1)
    return results

day11(data11, [2, 1000000])

Counter({1000000: 742305960572, 2: 9445168})